### Project : Analysing Music Records Sales using SQL

Chinook is a fictional music record store based out of the USA. The store offers a wide range of music genres, from classic rock and jazz to modern indie and hip-hop. With a passion for music and a commitment to providing exceptional customer experiences, Chinook aims to be the go-to destination for music lovers.


### To achieve so, they have the following business objectives:
1.	Establish Chinook as the premier destination for music records.

2.	Provide a diverse selection of music genres to cater to a wide range of customer preferences.

3.	Deliver excellent customer service to enhance the shopping experience.

4.	Build a loyal customer base and foster long-term relationships through engagement and promotional events.


In [1]:
import numpy as np 
import pandas as pd 
import sqlite3
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

database = 'chinook.db'

In [2]:
conn = sqlite3.connect(database)

tables = pd.read_sql("""SELECT *
                        FROM sqlite_master
                        WHERE type='table';
                        """, conn)
tables

,type,name,tbl_name,rootpage,sql
0,table,Album,Album,2,CREATE TABLE [Album]\n(\n [AlbumId] INTEGER...
1,table,Artist,Artist,3,CREATE TABLE [Artist]\n(\n [ArtistId] INTEG...
2,table,Customer,Customer,4,CREATE TABLE [Customer]\n(\n [CustomerId] I...
3,table,Employee,Employee,7,CREATE TABLE [Employee]\n(\n [EmployeeId] I...
4,table,Genre,Genre,9,CREATE TABLE [Genre]\n(\n [GenreId] INTEGER...
5,table,Invoice,Invoice,10,CREATE TABLE [Invoice]\n(\n [InvoiceId] INT...
6,table,InvoiceLine,InvoiceLine,12,CREATE TABLE [InvoiceLine]\n(\n [InvoiceLin...
7,table,MediaType,MediaType,14,CREATE TABLE [MediaType]\n(\n [MediaTypeId]...
8,table,Playlist,Playlist,15,CREATE TABLE [Playlist]\n(\n [PlaylistId] I...
9,table,PlaylistTrack,PlaylistTrack,16,CREATE TABLE [PlaylistTrack]\n(\n [Playlist...


### Segment 1:  Database - Tables, Columns, Relationships
**Identify the tables in the database and their respective columns.**

**Determine the number of records in each table within the schema.**

**Identify and handle any missing or inconsistent values in the dataset.**

**Analyse the data types of the columns in each table to ensure they are appropriate for the stored data.**


In [3]:
query1 = pd.read_sql("""
pragma table_info('customer');
""", conn)

query1

,cid,name,type,notnull,dflt_value,pk
0,0,CustomerId,INTEGER,1,None,1
1,1,FirstName,NVARCHAR(40),1,None,0
2,2,LastName,NVARCHAR(20),1,None,0
3,3,Company,NVARCHAR(80),0,None,0
4,4,Address,NVARCHAR(70),0,None,0
5,5,City,NVARCHAR(40),0,None,0
6,6,State,NVARCHAR(40),0,None,0
7,7,Country,NVARCHAR(40),0,None,0
8,8,PostalCode,NVARCHAR(10),0,None,0
9,9,Phone,NVARCHAR(24),0,None,0


### Segment 2: Track/Album Analysis
**How many tracks does Chinook store have in total**

In [4]:
query21 = pd.read_sql("""
SELECT count(Distinct Name)  as Total_No_Of_Track_Is
FROM Track;
""", conn)

query21

,Total_No_Of_Track_Is
0,3257


**Which tracks are the most popular among customers?**

In [5]:
quer22 = pd.read_sql("""
Select t.trackId, t.Name, count(il.TrackId) as No_of_sales
from Track t
inner join InvoiceLine il
on t.TrackId = il.TrackId
group by 1 , 2
order by 3 desc
limit 10
""", conn)
quer22

,TrackId,Name,No_of_sales
0,2,Balls to the Wall,2
1,8,Inject The Venom,2
2,9,Snowballed,2
3,20,Overdose,2
4,32,Deuces Are Wild,2
5,48,Not The Doctor,2
6,66,Por Causa De Você,2
7,84,Welcome Home (Sanitarium),2
8,161,Snowblind,2
9,162,Cornucopia,2


**What are the top-selling albums in the Chinook store?**

In [6]:
quer23 = pd.read_sql("""
SELECT
    a.AlbumId,
    a.Title AS AlbumTitle,
    COUNT(il.InvoiceLineId) AS NumTracksSold
FROM
    Album a
JOIN
    Track t ON a.AlbumId = t.AlbumId
JOIN
    InvoiceLine il ON t.TrackId = il.TrackId
GROUP BY
    a.AlbumId, a.Title
ORDER BY
    NumTracksSold DESC
LIMIT 10;

""",conn)
quer23

,AlbumId,AlbumTitle,NumTracksSold
0,23,Minha Historia,27
1,141,Greatest Hits,26
2,73,Unplugged,25
3,224,Acústico,22
4,37,Greatest Kiss,20
5,21,Prenda Minha,19
6,55,"Chronicle, Vol. 2",19
7,221,My Generation - The Very Best Of The Who,19
8,39,International Superhits,18
9,54,"Chronicle, Vol. 1",18


**Which artist or album has the highest number of tracks in the store?**

In [7]:
quer24 = pd.read_sql("""
SELECT
    a.ArtistId,
    a.Name AS ArtistName,
    al.AlbumId,
    al.Title AS AlbumTitle,
    COUNT(t.TrackId) AS NumTracks
FROM
    Artist a
LEFT JOIN
    Album al ON a.ArtistId = al.ArtistId
LEFT JOIN
    Track t ON al.AlbumId = t.AlbumId
GROUP BY
    a.ArtistId, a.Name, al.AlbumId, al.Title
ORDER BY
    NumTracks DESC
LIMIT 1;

""", conn)
quer24


,ArtistId,ArtistName,AlbumId,AlbumTitle,NumTracks
0,100,Lenny Kravitz,141,Greatest Hits,57


**-	Determine the number of tracks for each album and calculate the total duration of each album. Identify the albums that have the highest average track duration.**

In [8]:
quer25 = pd.read_sql("""
select 
a.albumid, a.title as Album_Title,count(t.trackid),
sum(t.Milliseconds)/60000 as DurationOfAlbum,
round(AVG(t.Milliseconds),2)/60000 AS AverageTrackDurationMilliseconds
from album a 
left join track t
on a.albumid = t.albumid
group by 1,2
order by 5 desc
""",conn)
quer25

,AlbumId,Album_Title,count(t.trackid),DurationOfAlbum,AverageTrackDurationMilliseconds
0,253,"Battlestar Galactica (Classic), Season 1",24,1170,48.759572
1,227,"Battlestar Galactica, Season 3",19,879,46.304422
2,229,"Lost, Season 3",26,1177,45.298450
3,231,"Lost, Season 2",24,1054,43.951133
4,226,Battlestar Galactica: The Story So Far,1,43,43.704167
...,...,...,...,...,...
342,328,"Charpentier: Divertissements, Airs & Concerts",1,1,1.837767
343,314,English Renaissance,2,3,1.691350
344,318,SCRIABIN: Vers la flamme,1,1,1.688217
345,345,Monteverdi: L'Orfeo,1,1,1.110650


**How does the genre distribution vary among albums?**

In [9]:
quer26 = pd.read_sql("""
SELECT
    a.AlbumId,
    a.Title AS AlbumTitle,
    COUNT(DISTINCT t.GenreId) AS NumberOfGenres,
    GROUP_CONCAT(DISTINCT g.Name) AS GenresList
FROM
    Album a
LEFT JOIN
    Track t ON a.AlbumId = t.AlbumId
LEFT JOIN
    Genre g ON t.GenreId = g.GenreId
GROUP BY
    a.AlbumId, a.Title
ORDER BY
    NumberOfGenres DESC;

""",conn)
quer26

,AlbumId,AlbumTitle,NumberOfGenres,GenresList
0,141,Greatest Hits,3,"Rock,Reggae,Metal"
1,227,"Battlestar Galactica, Season 3",3,"TV Shows,Science Fiction,Sci Fi & Fantasy"
2,73,Unplugged,2,"Blues,Latin"
3,102,Live After Death,2,"Heavy Metal,Metal"
4,109,Rock In Rio [CD2],2,"Rock,Metal"
...,...,...,...,...
342,343,Respighi:Pines of Rome,1,Classical
343,344,Schubert: The Late String Quartets & String Qu...,1,Classical
344,345,Monteverdi: L'Orfeo,1,Classical
345,346,Mozart: Chamber Music,1,Classical


In [10]:
query21 = pd.read_sql("""
select g.name,count(a.albumid) as NoofAlbum
from genre g
join track t
on t.genreid = g.genreid
join album a
on t.albumid = a.albumid
group by 1
order by 2 desc
""", conn)

query21

,Name,NoofAlbum
0,Rock,1297
1,Latin,579
2,Metal,374
3,Alternative & Punk,332
4,Jazz,130
5,TV Shows,93
6,Blues,81
7,Classical,74
8,Drama,64
9,R&B/Soul,61


### Segment 3: Customer Analysis


**Analyse customer demographics, purchase history, and preferences.**

In [11]:
quer31 = pd.read_sql("""
select *
from customer
""", conn)
quer31

,CustomerId,FirstName,LastName,Company,Address,City,State,Country,PostalCode,Phone,Fax,Email,SupportRepId
0,1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
1,2,Leonie,Köhler,None,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,+49 0711 2842222,None,leonekohler@surfeu.de,5
2,3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3
3,4,Bjørn,Hansen,None,Ullevålsveien 14,Oslo,None,Norway,0171,+47 22 44 22 22,None,bjorn.hansen@yahoo.no,4
4,5,František,Wichterlová,JetBrains s.r.o.,Klanova 9/506,Prague,None,Czech Republic,14700,+420 2 4172 5555,+420 2 4172 5555,frantisekw@jetbrains.com,4
5,6,Helena,Holý,None,Rilská 3174/6,Prague,None,Czech Republic,14300,+420 2 4177 0449,None,hholy@gmail.com,5
6,7,Astrid,Gruber,None,"Rotenturmstraße 4, 1010 Innere Stadt",Vienne,None,Austria,1010,+43 01 5134505,None,astrid.gruber@apple.at,5
7,8,Daan,Peeters,None,Grétrystraat 63,Brussels,None,Belgium,1000,+32 02 219 03 03,None,daan_peeters@apple.be,4
8,9,Kara,Nielsen,None,Sønder Boulevard 51,Copenhagen,None,Denmark,1720,+453 3331 9991,None,kara.nielsen@jubii.dk,4
9,10,Eduardo,Martins,Woodstock Discos,"Rua Dr. Falcão Filho, 155",São Paulo,SP,Brazil,01007-010,+55 (11) 3033-5446,+55 (11) 3033-4564,eduardo@woodstock.com.br,4


**How many unique customers are there in the Chinook store?**

In [12]:
quer32 = pd.read_sql("""
SELECT COUNT(DISTINCT CustomerId) AS UniqueCustomersCount
       FROM Customer;
""",conn)
quer32

,UniqueCustomersCount
0,59


**What is the distribution of customers across different countries?**

In [13]:
quer33 = pd.read_sql("""
SELECT country,
count(customerid) as NoOfCustomer
from customer
group by 1
order by 2  desc
limit 10;
""",conn)
quer33

,Country,NoOfCustomer
0,USA,13
1,Canada,8
2,France,5
3,Brazil,5
4,Germany,4
5,United Kingdom,3
6,Portugal,2
7,India,2
8,Czech Republic,2
9,Sweden,1


**Identify the average number of tracks purchased per customer for each genre**

In [14]:
quer34 = pd.read_sql("""
SELECT g.Name AS GenreName,
       COUNT(il.TrackId) / COUNT(DISTINCT c.CustomerId) AS AvgTracksPurchasedPerCustomer
FROM Genre g
JOIN Track t ON g.GenreId = t.GenreId
JOIN InvoiceLine il ON t.TrackId = il.TrackId
JOIN Invoice i ON il.InvoiceId = i.InvoiceId
JOIN Customer c ON i.CustomerId = c.CustomerId
GROUP BY g.Name
order by 2 desc
limit 10;

""",conn)
quer34

,GenreName,AvgTracksPurchasedPerCustomer
0,Rock,14
1,Latin,6
2,Metal,4
3,Alternative & Punk,4
4,Alternative,3
5,TV Shows,2
6,Soundtrack,2
7,Sci Fi & Fantasy,2
8,Reggae,2
9,R&B/Soul,2


**What is the average purchase value per customer?**

In [15]:
quer35 = pd.read_sql("""
SELECT round(AVG(total_purchase),2) AS AveragePurchaseValuePerCustomer
FROM (
    SELECT c.CustomerId, SUM(il.UnitPrice) AS total_purchase
    FROM Customer c
    JOIN Invoice i ON c.CustomerId = i.CustomerId
    JOIN InvoiceLine il ON i.InvoiceId = il.InvoiceId
    GROUP BY c.CustomerId
) AS customer_purchases;

""",conn)
quer35

,AveragePurchaseValuePerCustomer
0,39.47


**Which customers have made the highest number of purchases?**

In [16]:
quer36 = pd.read_sql("""
SELECT c.CustomerId, c.FirstName, c.LastName, COUNT(i.InvoiceId) AS PurchaseCount
FROM Customer c
JOIN Invoice i ON c.CustomerId = i.CustomerId
GROUP BY c.CustomerId, c.FirstName, c.LastName
ORDER BY PurchaseCount DESC;

""",conn)
quer36

,CustomerId,FirstName,LastName,PurchaseCount
0,1,Luís,Gonçalves,7
1,2,Leonie,Köhler,7
2,3,François,Tremblay,7
3,4,Bjørn,Hansen,7
4,5,František,Wichterlová,7
5,6,Helena,Holý,7
6,7,Astrid,Gruber,7
7,8,Daan,Peeters,7
8,9,Kara,Nielsen,7
9,10,Eduardo,Martins,7


### Segment 4: Support analysis


**What is the total sales revenue generated by each sales support agent?**



In [17]:
quer41 = pd.read_sql("""
SELECT e.EmployeeId, e.firstname||" "||e.lastname as FullName, SUM(il.UnitPrice) AS TotalSalesRevenue
FROM Employee e
JOIN Customer c ON e.EmployeeId = c.SupportRepId
JOIN Invoice i ON c.CustomerId = i.CustomerId
JOIN InvoiceLine il ON i.InvoiceId = il.InvoiceId
GROUP BY e.EmployeeId, e.FirstName, e.LastName;

""",conn)
quer41

,EmployeeId,FullName,TotalSalesRevenue
0,3,Jane Peacock,833.04
1,4,Margaret Park,775.40
2,5,Steve Johnson,720.16


**What is the average order value for each sales support agent?**

In [18]:
quer42 = pd.read_sql("""
SELECT e.EmployeeId, e.firstname||" "||e.lastname as FullName, 
       round(AVG(i.Total),2) AS AverageOrderValue
FROM Employee e
JOIN Customer c ON e.EmployeeId = c.SupportRepId
JOIN Invoice i ON c.CustomerId = i.CustomerId
GROUP BY e.EmployeeId, e.FirstName, e.LastName;

""",conn)
quer42

,EmployeeId,FullName,AverageOrderValue
0,3,Jane Peacock,5.71
1,4,Margaret Park,5.54
2,5,Steve Johnson,5.72


**Is there any significant difference in sales performance between agents reporting to different managers?**

In [19]:
quer43 = pd.read_sql("""
SELECT e2.EmployeeId AS ManagerId, e2.FirstName||" "|| e2.lastname AS ManagerName,
       e1.EmployeeId AS AgentId, e1.FirstName||" "|| e1.lastname AS AgentName,
       AVG(i.Total) AS AverageOrderValue
FROM Employee e1
JOIN Employee e2 ON e1.ReportsTo = e2.EmployeeId
JOIN Customer c ON e1.EmployeeId = c.SupportRepId
JOIN Invoice i ON c.CustomerId = i.CustomerId
GROUP BY e2.EmployeeId, e2.FirstName, e2.LastName, e1.EmployeeId, e1.FirstName, e1.LastName;

""",conn)
quer43

,ManagerId,ManagerName,AgentId,AgentName,AverageOrderValue
0,2,Nancy Edwards,3,Jane Peacock,5.705753
1,2,Nancy Edwards,4,Margaret Park,5.538571
2,2,Nancy Edwards,5,Steve Johnson,5.715556


**How does the hiring date of sales support agents impact their sales performance?**

In [20]:
quer44 = pd.read_sql("""
SELECT e.EmployeeId,e.firstname ||" "||e.lastname as FullName, e.HireDate,
       AVG(i.Total) AS AverageOrderValue
FROM Employee e
JOIN Customer c ON e.EmployeeId = c.SupportRepId
JOIN Invoice i ON c.CustomerId = i.CustomerId
GROUP BY e.EmployeeId, e.FirstName, e.LastName, e.HireDate;

""",conn)
quer44

,EmployeeId,FullName,HireDate,AverageOrderValue
0,3,Jane Peacock,2002-04-01 00:00:00,5.705753
1,4,Margaret Park,2003-05-03 00:00:00,5.538571
2,5,Steve Johnson,2003-10-17 00:00:00,5.715556


### Segment 5: Sales Analysis


**What is the total revenue generated by the Chinook store?**


In [21]:
quer51 = pd.read_sql("""
select sum(unitprice*quantity) as TotalRevenue
from invoiceline
""",conn)
quer51

,TotalRevenue
0,2328.6


**How does the revenue vary across different countries?**

In [22]:
quer52 = pd.read_sql("""
select i.billingcountry as Country,sum(il.unitprice*il.quantity) as Revenue
from invoiceline il
right join invoice i
on il.invoiceid = i.invoiceid
group by 1
""",conn)
quer52

,Country,Revenue
0,Argentina,37.62
1,Australia,37.62
2,Austria,42.62
3,Belgium,37.62
4,Brazil,190.10
5,Canada,303.96
6,Chile,46.62
7,Czech Republic,90.24
8,Denmark,37.62
9,Finland,41.62


**Calculate the percentage of total sales contributed by each genre in the USA and identify the best-selling genres and artists.**

In [23]:
quer53 = pd.read_sql("""
select g.name as Genre,
a.name as Artist,
SUM(il.Quantity * il.UnitPrice) AS TotalSales,
    ROUND(SUM(il.Quantity * il.UnitPrice) / (SELECT SUM(Quantity * UnitPrice) FROM InvoiceLine)* 100, 2) AS PercentageOfTotalSales
from genre g
inner join track t
on g.genreid = t.genreid
inner join invoiceline il
on t.trackid = il.trackid
inner join invoice i
on il.invoiceid = i.invoiceid
inner join album al
on al.albumid = t.albumid
inner join artist a
on a.artistid = al.artistid
where i.billingcountry= 'USA'
group by 1,2
order by 3 desc
""",conn)

quer53

,Genre,Artist,TotalSales,PercentageOfTotalSales
0,Metal,Metallica,26.73,1.15
1,Rock,U2,22.77,0.98
2,TV Shows,The Office,17.91,0.77
3,Comedy,The Office,15.92,0.68
4,Metal,Iron Maiden,14.85,0.64
...,...,...,...,...
121,Rock,Santana,0.99,0.04
122,Rock,"Terry Bozzio, Tony Levin & Steve Stevens",0.99,0.04
123,Rock,Velvet Revolver,0.99,0.04
124,Soundtrack,Gilberto Gil,0.99,0.04


**Evaluate sales performance over time to identify seasonal trends and peak periods.**

In [24]:
quer54 = pd.read_sql("""
select date(i.invoicedate) as Date, sum(il.quantity*il.unitprice) as TotalSales
from invoice i
inner join invoiceline il
on i.invoiceid = il.invoiceid
group by 1
""",conn)
quer54

,Date,TotalSales
0,2009-01-01,1.98
1,2009-01-02,3.96
2,2009-01-03,5.94
3,2009-01-06,8.91
4,2009-01-11,13.86
...,...,...
349,2013-12-05,3.96
350,2013-12-06,5.94
351,2013-12-09,8.91
352,2013-12-14,13.86


**Determine the average transaction value and customer lifetime value to gauge business profitability.**

In [25]:
query55 = pd.read_sql("""
select c.customerid, avg(i.total) as AvgTransaction, sum(i.total) as LifetimeValue
from customer c
join invoice i
on c.customerid = i.customerid
group by 1;
""", conn)

query55

,CustomerId,AvgTransaction,LifetimeValue
0,1,5.660000,39.62
1,2,5.374286,37.62
2,3,5.660000,39.62
3,4,5.660000,39.62
4,5,5.802857,40.62
5,6,7.088571,49.62
6,7,6.088571,42.62
7,8,5.374286,37.62
8,9,5.374286,37.62
9,10,5.374286,37.62


### Segment 6: SQL Concepts
**Use subqueries to find the top-selling tracks in the USA and identify their associated genres.**


In [26]:
query61 = pd.read_sql("""
SELECT
    t.Name AS TrackName,
    g.Name AS GenreName,
    (SELECT COUNT(*) FROM InvoiceLine il
     JOIN Invoice i ON il.InvoiceId = i.InvoiceId
     JOIN Customer c ON i.CustomerId = c.CustomerId
     WHERE il.TrackId = t.TrackId AND c.Country = 'USA') AS TotalSales
FROM
    Track t
    JOIN Genre g ON t.GenreId = g.GenreId
    where totalsales != 0
ORDER BY
    TotalSales DESC

""", conn)

query61

,TrackName,GenreName,TotalSales
0,Meditação,Latin,2
1,Train In Vain,Alternative & Punk,2
2,Speed Of Light,Rock,2
3,Sun King,Rock,2
4,Música No Ar,Latin,2
...,...,...,...
481,Amy Amy Amy (Outro),Pop,1
482,"A Midsummer Night's Dream, Op.61 Incidental Mu...",Classical,1
483,Symphony No. 3 Op. 36 for Orchestra and Sopran...,Classical,1
484,Symphony No. 2: III. Allegro vivace,Classical,1


**Implement joins to retrieve customer information along with their preferred genres and country of origin.**

In [27]:
query62 = pd.read_sql("""
SELECT c.CustomerId, c.FirstName, c.LastName, c.Country, g.Name AS PreferredGenre
FROM
    Customer AS c
    LEFT JOIN Invoice AS i ON c.CustomerId = i.CustomerId
    LEFT JOIN InvoiceLine AS il ON i.InvoiceId = il.InvoiceId
    LEFT JOIN Track AS t ON il.TrackId = t.TrackId
    LEFT JOIN Genre AS g ON t.GenreId = g.GenreId
    
GROUP BY 1;
""", conn)

query62

,CustomerId,FirstName,LastName,Country,PreferredGenre
0,1,Luís,Gonçalves,Brazil,Sci Fi & Fantasy
1,2,Leonie,Köhler,Germany,Rock
2,3,François,Tremblay,Canada,TV Shows
3,4,Bjørn,Hansen,Norway,Rock
4,5,František,Wichterlová,Czech Republic,Rock
5,6,Helena,Holý,Czech Republic,Rock
6,7,Astrid,Gruber,Austria,Metal
7,8,Daan,Peeters,Belgium,Rock
8,9,Kara,Nielsen,Denmark,Metal
9,10,Eduardo,Martins,Brazil,Latin


**Apply window functions to rank genres based on their sales performance in the USA.**

In [28]:
query62 = pd.read_sql("""
select  rank() over(order by TotalQuantitySale desc) as Rank, *
from (select g.name as GenreName,count(il.quantity) as TotalQuantitySale
from invoiceline il
join track t on t.trackid = il.trackid
join genre g on g.genreid = t.genreid
join invoice i on i.invoiceid = il.invoiceid
where i.billingcountry = 'USA'
group by g.name
);
""", conn)

query62

,Rank,GenreName,TotalQuantitySale
0,1,Rock,157
1,2,Latin,91
2,3,Metal,64
3,4,Alternative & Punk,50
4,5,Jazz,22
5,6,Blues,15
6,7,TV Shows,14
7,8,R&B/Soul,12
8,9,Classical,8
9,9,Comedy,8


**Create stored procedures to calculate the average revenue per sale for each genre in the USA.**

In [29]:
query63 = pd.read_sql("""
select g.name as Genre, avg(il.quantity*il.unitprice) as AvgRevnue
from invoiceline il
join track t on t.trackid = il.trackid
join genre g on g.genreid = t.genreid
join invoice i on i.invoiceid = il.invoiceid
where i.billingcountry = 'USA'
group by g.name
order by 2 desc;
""", conn)

query63

,Genre,AvgRevnue
0,Science Fiction,1.99
1,Drama,1.99
2,Comedy,1.99
3,Sci Fi & Fantasy,1.99
4,TV Shows,1.99
5,Metal,0.99
6,Alternative & Punk,0.99
7,Reggae,0.99
8,R&B/Soul,0.99
9,Classical,0.99


### Segment 7: Market Expansion and Advertising Strategy


**Recommend the three albums from the new record label that should be prioritised for advertising and promotion in the USA based on genre sales analysis.**


In [30]:
query71 = pd.read_sql("""
SELECT
    a.Title AS AlbumTitle,
    g.Name AS GenreName,
    SUM(il.Quantity) AS TotalSales
FROM
    Album AS a
    JOIN Track AS t ON a.AlbumId = t.AlbumId
    JOIN Genre AS g ON t.GenreId = g.GenreId
    JOIN InvoiceLine AS il ON t.TrackId = il.TrackId
    JOIN Invoice AS i ON il.InvoiceId = i.InvoiceId
    JOIN Customer AS c ON i.CustomerId = c.CustomerId
WHERE
    c.Country = 'USA'
GROUP BY
    a.AlbumId
ORDER BY 3 desc;

""", conn)

query71

,AlbumTitle,GenreName,TotalSales
0,"The Office, Season 3",Comedy,14
1,Unplugged,Blues,11
2,Prenda Minha,Latin,11
3,Chill: Brazil (Disc 2),Latin,10
4,Back to Black,R&B/Soul,9
...,...,...,...
200,Black Sabbath,Metal,1
201,Audioslave,Rock,1
202,Plays Metallica By Four Cellos,Metal,1
203,Facelift,Rock,1


**Identify high-value customers and develop personalised marketing strategies to nurture customer loyalty.**

In [31]:
query72 = pd.read_sql("""
SELECT
    c.CustomerId,
    c.FirstName ||" "||c.LastName as FullName,
    SUM(il.UnitPrice * il.Quantity) AS TotalRevenue,
    COUNT(DISTINCT i.InvoiceId) AS TotalPurchases,
    AVG(il.UnitPrice * il.Quantity) AS AverageOrderValue
FROM
    Customer AS c
    JOIN Invoice AS i ON c.CustomerId = i.CustomerId
    JOIN InvoiceLine AS il ON i.InvoiceId = il.InvoiceId
GROUP BY
    c.CustomerId
HAVING
    TotalRevenue > 10
ORDER BY
    TotalRevenue DESC;
""", conn)

query72

,CustomerId,FullName,TotalRevenue,TotalPurchases,AverageOrderValue
0,6,Helena Holý,49.62,7,1.305789
1,26,Richard Cunningham,47.62,7,1.253158
2,57,Luis Rojas,46.62,7,1.226842
3,46,Hugh O'Reilly,45.62,7,1.200526
4,45,Ladislav Kovács,45.62,7,1.200526
5,37,Fynn Zimmermann,43.62,7,1.147895
6,24,Frank Ralston,43.62,7,1.147895
7,28,Julia Barnett,43.62,7,1.147895
8,25,Victor Stevens,42.62,7,1.121579
9,7,Astrid Gruber,42.62,7,1.121579


**Analyse the potential for expanding sales of different genres in other countries based on their sales performance in the USA.**

In [32]:
query73 = pd.read_sql("""
SELECT
    g.Name AS GenreName,
    SUM(il.Quantity) AS TotalSales
FROM
    Genre AS g
    JOIN Track AS t ON g.GenreId = t.GenreId
    JOIN InvoiceLine AS il ON t.TrackId = il.TrackId
    JOIN Invoice AS i ON il.InvoiceId = i.InvoiceId
    JOIN Customer AS c ON i.CustomerId = c.CustomerId
WHERE
    c.Country = 'USA'
GROUP BY
    g.GenreId
""", conn)

query73

,GenreName,TotalSales
0,Rock,157
1,Jazz,22
2,Metal,64
3,Alternative & Punk,50
4,Rock And Roll,3
5,Blues,15
6,Latin,91
7,Reggae,6
8,Pop,5
9,Soundtrack,4


**Determine the top-selling genres in countries other than the USA and identify any commonalities or differences.**

In [33]:
query74 = pd.read_sql("""
SELECT
    c.Country,
    g.Name AS GenreName,
    SUM(il.Quantity) AS TotalSales
FROM
    Genre AS g
    JOIN Track AS t ON g.GenreId = t.GenreId
    JOIN InvoiceLine AS il ON t.TrackId = il.TrackId
    JOIN Invoice AS i ON il.InvoiceId = i.InvoiceId
    JOIN Customer AS c ON i.CustomerId = c.CustomerId
WHERE
    c.Country <> 'USA' -- Exclude the USA
GROUP BY
    c.Country,g.name
    order by 3 desc

""", conn)

query74

,Country,GenreName,TotalSales
0,Canada,Rock,107
1,Brazil,Rock,81
2,France,Rock,65
3,Germany,Rock,62
4,Canada,Latin,60
...,...,...,...
210,Spain,Hip Hop/Rap,1
211,Sweden,Jazz,1
212,Sweden,TV Shows,1
213,Sweden,World,1
